In [37]:
import requests
from bs4 import BeautifulSoup
import json
import urllib.request
from urllib.parse import urlparse
import pandas as pd

In [96]:
DOMAIN = 'https://zh.wikipedia.org'

category = {
    '上市':'%E8%87%BA%E7%81%A3%E8%AD%89%E5%88%B8%E4%BA%A4%E6%98%93%E6%89%80%E4%B8%8A%E5%B8%82%E5%85%AC%E5%8F%B8',
    '上櫃':'%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E8%AD%89%E5%88%B8%E6%AB%83%E6%AA%AF%E8%B2%B7%E8%B3%A3%E4%B8%AD%E5%BF%83%E4%B8%8A%E6%AB%83%E5%85%AC%E5%8F%B8',
}

company_list = []
stock_num_list =[]
company_num_list = []
sameword_list = []

for cat, cat_id in category.items():
    url = 'https://zh.wikipedia.org/wiki/Category:' + cat_id
    resp = requests.get(url)
    doc = BeautifulSoup(resp.text, 'lxml')
    
    if cat == '上市':
        origin = doc.select('.mw-category-group')[3:]
    else:
        origin = doc.select('.mw-category-group')[1:]
    
    # 先到category頁面，再個別爬公司資訊
    for news in origin:
        for content in news.select('li a'):
            pg_url = DOMAIN + content['href']
            resp2 = requests.get(pg_url)
            doc2 = BeautifulSoup(resp2.text, 'lxml') 

            if re.search(r'股票代號', str(doc2.select('.infobox'))):
                try:
                    box = doc2.select('.infobox')[0].text.split('股票代號')[1]
                except:
                    box = doc2.select('.infobox')[1].text.split('股票代號')[1]
                stock_num = re.search(r'\d{4}', box).group(0)
            else:
                stock_num = "None"
        
            if re.search(r'統一編號', str(doc2.select('.infobox'))):
                try:
                    box = doc2.select('.infobox')[0].text.split('統一編號')[1]
                except:
                    box = doc2.select('.infobox')[1].text.split('統一編號')[1]
                company_num = re.search(r'\d{8}', box).group(0)
            else:
                company_num = "None"
        
            for i in doc2.select('#mw-content-text .mw-parser-output p')[:1]:
                for keyword in i.select('b'):
                    if not re.search(r'\d{4}', keyword.text):
                        company_list.append(content['title'])
                        stock_num_list.append(stock_num)
                        company_num_list.append(company_num)
                        sameword_list.append(keyword.text.strip())

df = pd.DataFrame({'公司股票號碼':stock_num_list, '公司名稱':company_list, '公司統一編號': company_num_list, '公司別名': sameword_list})
df.to_csv('stock_info.csv', encoding= 'utf_8_sig')

,公司股票號碼,公司名稱,公司統一編號,公司別名
0,5830,三信商業銀行,None,三信商業銀行
1,5830,三信商業銀行,None,三信商銀
2,5830,三信商業銀行,None,三信
3,3682,亞太電信,70771579,亞太電信股份有限公司
4,3682,亞太電信,70771579,Asia Pacific Telecom
5,3682,亞太電信,70771579,APT
6,3682,亞太電信,70771579,亞太電
7,5852,元大商業銀行,None,元大銀行
8,2881,台北富邦商業銀行,None,台北富邦商業銀行
9,2881,台北富邦商業銀行,None,台北富邦銀行
